## Day 78 Lecture 2 Assignment

In this assignment, we will learn about dataframes in Spark.

Run the cells below to start a spark session.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 59kB/s 
     |████████████████████████████████| 204kB 43.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=f69cea948b90141a103790f5cf546907d544e859605f06a2013b9dfb0628cbdb
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark.sql import SparkSession

In [0]:
APP_NAME = "Day78"
SPARK_URL = "local[*]"

In [0]:
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [0]:
video = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/Video_Games_Sales_as_at_22_Dec_2016.csv")

In [0]:
video.show()

+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|                Name|Platform|Year_of_Release|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|          Developer|Rating|
+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|          Wii Sports|     Wii|           2006|      Sports|            Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322|           Nintendo|     E|
|   Super Mario Bros.|     NES|           1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        null|        null|     

How many unique publishers are there? 

In [0]:
# Answer below:
video.select('Publisher').distinct().count()


582

In [0]:
video.count()

16719

How many games has each publisher released?

In [0]:
# Answer below:
video.groupBy('Publisher').count().show()


+--------------------+-----+
|           Publisher|count|
+--------------------+-----+
|        Funbox Media|    5|
|         Media Rings|    3|
| Iceberg Interactive|    3|
|         Tigervision|    3|
|   bitComposer Games|    5|
|                 3DO|   36|
|           Telegames|    8|
|   Jack of All Games|    3|
|Nihon Falcom Corp...|    7|
|               Sting|    9|
|         id Software|    1|
|        IE Institute|    5|
|           Game Life|    2|
| Karin Entertainment|    1|
|     Evolution Games|    1|
|              Takuyo|    1|
|          Infogrames|   62|
|  Illusion Softworks|    1|
|          Enterbrain|   15|
|                 Elf|    2|
+--------------------+-----+
only showing top 20 rows



What is the total global sales per publisher? Make sure to get rid of NAs.

In [0]:
from pyspark.sql.functions import UserDefinedFunction

In [0]:
removeNan = UserDefinedFunction (lambda x: 0 if x == 'null' else float(x) )
new_col = video.withColumn('Global_Sales_noNan', removeNan(video['Global_Sales']))

In [0]:
# Answer below:
video.dropna().groupBy('Publisher').sum('Global_Sales').sort('sum(Global_Sales)', ascending=False).show()


+--------------------+------------------+
|           Publisher| sum(Global_Sales)|
+--------------------+------------------+
|     Electronic Arts| 885.4599999999996|
|            Nintendo| 850.1000000000004|
|          Activision| 540.7199999999997|
|Sony Computer Ent...| 388.0999999999992|
|Take-Two Interactive|350.40999999999974|
|             Ubisoft|  341.219999999999|
|Microsoft Game St...|216.49000000000015|
|                 THQ| 162.9700000000005|
|                Sega|150.38999999999984|
|Konami Digital En...|139.39000000000013|
|  Namco Bandai Games|118.61999999999998|
|Warner Bros. Inte...|118.42000000000004|
|              Capcom|113.55999999999996|
|         Square Enix|107.43000000000002|
|               Atari| 84.19999999999999|
|           LucasArts| 74.12999999999998|
|  Bethesda Softworks|             71.41|
|   Eidos Interactive| 47.24000000000006|
|        Midway Games| 44.70000000000002|
|       Vivendi Games|             43.94|
+--------------------+------------

What is the name of the game with the highest sales in North America?

In [0]:
# Answer below:
game_name_top =  video.select('Name', 'NA_sales').sort('NA_sales',ascending=False)


In [0]:
type(game_name_top.head().Name)

str

In [0]:
game_name_top.toPandas()

,Name,NA_sales
0,Wii Sports,41.36
1,Super Mario Bros.,29.08
2,Duck Hunt,26.93
3,Tetris,23.20
4,Mario Kart Wii,15.68
...,...,...
16714,SCORE International Baja 1000: The Official Game,0.00
16715,Samurai Warriors: Sanada Maru,0.00
16716,LMA Manager 2007,0.00
16717,Haitaka no Psychedelica,0.00


How many rows are in the data set? How many rows are left after we drop all nulls?

In [0]:
# Answer below:
video.count()


16719

In [0]:
video.dropna().count()

6947

For each game, compute the percent NA sales out of the global sales.

In [0]:
# Answer below:
#removeNan = UserDefinedFunction (lambda x: 0 if x == 'null' else float(x) )

#new_col = video.withColumn('Global_Sales_noNan', removeNan(video['Global_Sales']))
video2 = video.withColumn('NA_%', video['NA_sales']/video['Global_Sales'])

In [0]:
video2.show()

+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+-------------------+
|                Name|Platform|Year_of_Release|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|          Developer|Rating|               NA_%|
+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+-------------------+
|          Wii Sports|     Wii|           2006|      Sports|            Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322|           Nintendo|     E| 0.5011510965709439|
|   Super Mario Bros.|     NES|           1985|    Platform|            Nintendo|   

Write a udf that returns `Yes` if a value is over 0.5 and `No` otherwise. Apply this function to the NA percent column and create a new column that tells us whether NA sales made up more than 50% of a game's total sales.

In [0]:
# Answer below:
udf = UserDefinedFunction(lambda x: 'Yes' if x>0.5 else 'No')
video2.withColumn('NA_bool', udf("NA_%")).show()

+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+-------------------+-------+
|                Name|Platform|Year_of_Release|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|          Developer|Rating|               NA_%|NA_bool|
+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+-------------------+-------+
|          Wii Sports|     Wii|           2006|      Sports|            Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322|           Nintendo|     E| 0.5011510965709439|    Yes|
|   Super Mario Bros.|     NES|           1985|    P